# Pandas API

źródło: https://www.sicara.fr/blog-technique/run-pandas-code-on-spark, 


Pandas to bardzo potężna biblioteka, którą znają wszyscy analitycy danych, ale kod Pandas może działać tylko na jednej jednostce. W związku z tym jeżeli przetwarzamy duży zestaw danych za pomocą pandas będzie się to działo bardzo wolno i najprawdopodobniej pojawi się OOM error.

Zwykle wtedy do gry wchodzi Spark. PySpark co prawda zawiera  moduł o nazwie Spark SQL, który zapewnia obiekty typu DataFrame podobny do ramek pandas, ale mają one wady:
- napisany wcześniej kod pandas nie może zostać poniewnie użyty, ponieważ Pandas nie jest kompatybilny z PySpark DataFrames.
- składnia PySpark bardzo różni się od składni Pandas (patrz poniżej), co utrudnia PySparka osobom pracującyjm wcześniej w pandas

Poniżej przykład różnic:


In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
# Może chwilę potrwać
spark = SparkSession.builder.appName("UczymySięSparka").getOrCreate()
spark

'''
#PySpark:

df = spark.read.option("inferSchema", True).cvs("data.csv")
df = df.toDF("x","y", "z")
df = df.withColumn("x2", df.x * df.x)

#Pandas: 
df = pd.read_csv("data.csv")
df.columns = ["x","y","z"]
df["x2"] = df.x * df.x

'''


'\n#PySpark:\n\ndf = spark.read.option("inferSchema", True).cvs("data.csv")\ndf = df.toDF("x","y", "z")\ndf = df.withColumn("x2", df.x * df.x)\n\n#Pandas: \ndf = pd.read_csv("data.csv")\ndf.columns = ["x","y","z"]\ndf["x2"] = df.x * df.x\n\n'

24 kwietnia 2019 r. firma Databricks ogłosiła nowy projektopen source o nazwie Koalas, którego celem było udostępnienie interfejsu API Pandas na platformie Spark. Biblioteka nabrała rozpędu i została oficjalnie połączona z PySpark w Spark 3.2 (październik 2021 r.) i nazwana API Pandas on Spark.

Interfejs API Pandas ma taką samą składnię jak Pandas, ale "pod spodem" używa ramek PySparkowych. Oznacza to, że kod napisany za pomocą Pandas API może być uruchamiany w systemie master - slave, w których Spark jest skonfigurowany (w przeciwieństwie do Pandas), co pozwala na obsługę dużych zbiorów danych. Korzystająz z Pandas API można robić prawie wszystko, co z ramkami Pandowymi (~83% funkcji dostępnych w pyspark.pandas). Pełna lista funkcji: 

https://spark.apache.org/docs/latest/api/python//reference/pyspark.pandas/general_functions.html

### Co zrobić, jeśli nie mogę znaleźć funkcji Pandas w Pandas API?

Ponieważ ramka danych Pandas-on-Spark wykorzystuje ramkę danych PySpark "pod spodem", można ją przekonwertować z/do ramki danych PySpark. Dlatego jeśli nie możesz znaleźć potrzebnej funkcji, nadal możesz wykonać następujące czynności:

* przekształcić ramkę danych Pandas-on-Spark w ramkę danych PySpark
* wykonać tyle transformacji ile potrzeba za pomocą PySpark
* przekonwertować ramkę danych PySpark z powrotem na ramkę danych Pandas-on-Spark

In [2]:
import pyspark.pandas as ps
psdf = ps.range(10)
sdf = psdf.to_spark().filter("id > 5")
sdf.show()

+---+
| id|
+---+
|  6|
|  7|
|  8|
|  9|
+---+



In [3]:
sdf.to_pandas_on_spark()

,id
0,6
1,7
2,8
3,9


Poniżej więcej przykładów jak wymiennie korzystać z PySparka i pandas:

In [17]:
import pyspark.pandas as ps # przekstzałcenie na ramkę pandasową
psdf = ps.range(10)
pdf = psdf.to_pandas()
pdf.values

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]], dtype=int64)

In [18]:
ps.from_pandas(pdf) # pandas on spark 


,id
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [19]:
import pyspark.pandas as ps

psdf = ps.range(10)
sdf = psdf.to_spark().filter("id > 5") ## ramki sprakowe i pandasowe są do siebie bardzo podobne, tu przekształcenie z ramki pandasowej na sparkową
sdf.show()

+---+
| id|
+---+
|  6|
|  7|
|  8|
|  9|
+---+



In [22]:
sdf.to_pandas_on_spark() ## ze sparka do pandas on spark

,id
0,6
1,7
2,8
3,9


# Jak działa Pandas API?
Gdy użytkownik tworzy ramkę danych Pandas-on-Spark, powstaje wtedy też „ramka wewnętrzna” i ramka sparkowa.

„Ramka wewnętrzna” zapewnia konwersje między ramkami pandas-on-Spark i PySpark. Przechowuje metadane, takie jak mapowanie danych w kolumnach i indeksów.

Pozwala ramce Pandas-on-Spark na obsługę funkcji Pandas, które nie są obsługiwane przez ramki PySpark:

* mutable syntax: dzięki czemu nie trzeba tworzyć nowej ramki za każdym razem, gdy chcemy coś zmodyfikować
* indeks sekwencyjny: żeby można było manipulować ramką w oparciu o indeks (patrz poniżej)
* pandowe typy danych

Należy pamiętać, że dane są rozproszone po wielu workerach, podczas gdy w Pandzie dane pozostają na jednej maszynie.

### Indeksowanie

W przeciwieństwie do ramki PySpark, ramki Pandas-on-Spark replikują funkcjonalność indeksowania Pandas (dzięki wspomianej wyżej wewnętrznej ramce). Dla przypomnienia, indeksy służą do uzyskiwania dostępu do wierszy przez indeksatory loc/iloc lub do mapowania właściwych wierszy w przypadku operacji łączących dwie ramki danych lub serie (na przykład df1 + df2) i tak dalej.

Jeśli żadna z kolumn nie została określona jako indeks zostanie użyty indeks domyślny. Może to być jeden z 3:

**Sekwencja** </br>
Używany domyślnie. Implementuje sekwencję, która zwiększa się o jeden razem z każdym kolejnym rekordem. Najprawdopodobniej spowoduje to przeniesienie całej ramki do jednego klastra, co będzie bardzo powolne i najprawdopodoniej rzuci OOM error. Nie należy go używać, gdy zbiór danych jest duży.

In [4]:
import pyspark.pandas as ps
ps.set_option('compute.default_index_type', 'sequence')
psdf = ps.range(3)
ps.reset_option('compute.default_index_type')
psdf.index


Int64Index([0, 1, 2], dtype='int64')

**Rozproszona sekwencja** </br>
Mechanizm jest ten sam, co wyżej, ale indeks rozproszony można wykorzystywać razem z partycjonowaniem. Powinien zostać użyty jeśli zbiór  danych jest duży i potrzebny jest indeks sekwencyjny. Należy zauważyć, że jeśli po utworzeniu tego indeksu do zbioru danych zostanie dodanych więcej rekordów nie ma gwarancji, że indeks pozostanie sekwencyjny.



In [5]:
import pyspark.pandas as ps
ps.set_option('compute.default_index_type', 'distributed-sequence')
psdf = ps.range(3)
ps.reset_option('compute.default_index_type')
psdf.index

Int64Index([0, 1, 2], dtype='int64')

**Rozlokowany (distributed)** </br>
Implementuje ciąg rosnący monotonicznie, ale nie nieprzerwany (np. 1, 8, 12), wartości są przypadkowe. Nie ma nic wspólnego z indeksowaniem Pandas. Pod względem wydajności jest najlepszy, ale nie można go używać do wykonywania operacji na dwóch ramkach. 

Poniżej przykład ustawiania typu indeksu, z któego chcemy skorzystać: 

In [6]:
import pyspark.pandas as ps
ps.set_option('compute.default_index_type', 'distributed')
psdf = ps.range(3)
ps.reset_option('compute.default_index_type')
psdf.index


Int64Index([42949672960, 85899345920, 128849018880], dtype='int64')

### Pozostałe opcje

Pandas API ma system opcji, który pozwala dostosować niektóre aspekty jego pracy. Najczęściej użytkownicy zmieniają opcję wyświetlania wyników. Najważniejsze w tym wypadku są dwa polecenia: 

* get_option() / set_option() - podejrzenie/ustawienie opcji
* reset_option() - zresteowanie danej opcji

In [7]:
import pyspark.pandas as ps
ps.get_option('compute.max_rows')

1000

In [8]:
ps.set_option('compute.max_rows', 2000)
ps.get_option('compute.max_rows')

2000

In [11]:
ps.reset_option("display.max_rows")


In [12]:
ps.get_option('compute.max_rows')

2000

### Operacje na różnych ramkach
Pandas API domyślnie blokuje operacje na różnych ramkach (lub seriach), aby zapobiec zbyt ciężkim obliczeniowo operacjom. 

Można to włączyć, ustawiając compute.ops_on_diff_frames na True. 

In [13]:
import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)
psdf1 = ps.range(5)
psdf2 = ps.DataFrame({'id': [5, 4, 3]})
(psdf1 - psdf2).sort_index()

,id
0,-5.0
1,-3.0
2,-1.0
3,NaN
4,NaN


In [15]:
ps.reset_option('compute.ops_on_diff_frames')

In [16]:
import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)
psdf = ps.range(5)
psser_a = ps.Series([1, 2, 3, 4])
psdf['new_col'] = psser_a
psdf

,id,new_col
0,0,1.0
1,1,2.0
2,2,3.0
3,3,4.0
4,4,NaN


Wszystkie dostępne opcje można sprawdzić pod tym linkiem: https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/options.html#available-options